In [1]:
# PART 2: Narrow travel searches based on temperature and precipitation
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import API key
from configure import g_key

In [13]:
# 2. Read and load the cities database
city_data_df = pd.read_csv('weather_data\challenge_data\WeatherPy_challenge.csv')
city_data_df.head(3)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Pavlovka,RU,2019-11-08 05:15:31,52.69,47.13,43.95,86,100,12.53,light rain,0.81,0.0
1,1,Vilhena,BR,2019-11-08 05:15:31,-12.74,-60.15,68.54,98,100,2.80,light rain,0.94,0.0
2,2,Mehamn,NO,2019-11-08 05:13:24,71.03,27.85,30.20,92,40,10.29,scattered clouds,0.00,0.0


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# 3. Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer if he/she would like it to be raining or not
rain = input('Do you want it to be raining? (yes/no)')

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88
Do you want it to be raining? (yes/no)no


In [5]:
# Filter dataset to find cities that fit the temperature criteria
favored_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
# Filter dataset to fit raining criteria
if (rain == 'yes')|(rain == 'Yes')|(rain == 'YES') :
    preferred_cities_df = favored_cities_df.loc[(favored_cities_df['Rain inches (last 3 hrs)'] > 0) | (favored_cities_df['Snow inches (last 3 hrs)'] > 0)]
else:
    preferred_cities_df = favored_cities_df.loc[(favored_cities_df['Rain inches (last 3 hrs)'] == 0) & (favored_cities_df['Snow inches (last 3 hrs)'] == 0)]
    
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
10,10,Hervey Bay,AU,2019-11-08 05:13:53,-25.30,152.85,86.00,48,0,25.28,clear sky,0.0,0.0
24,24,Thinadhoo,MV,2019-11-08 05:15:36,0.53,72.93,82.49,72,90,9.48,overcast clouds,0.0,0.0
40,40,Acapulco,MX,2019-11-08 05:11:09,16.86,-99.88,80.60,88,40,4.70,scattered clouds,0.0,0.0
42,42,Cockburn Town,TC,2019-11-08 05:15:40,21.46,-71.14,82.22,79,3,16.37,clear sky,0.0,0.0
43,43,Hithadhoo,MV,2019-11-08 05:14:28,-0.60,73.08,82.47,68,0,6.20,clear sky,0.0,0.0


In [6]:
# Create dataframe called hotel_df to store hotel names along with city, country, max temperature and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp','Current Description', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Hervey Bay,AU,86.00,clear sky,-25.30,152.85,
24,Thinadhoo,MV,82.49,overcast clouds,0.53,72.93,
40,Acapulco,MX,80.60,scattered clouds,16.86,-99.88,
42,Cockburn Town,TC,82.22,clear sky,21.46,-71.14,
43,Hithadhoo,MV,82.47,clear sky,-0.60,73.08,


In [7]:
# Set the parameters to search for a hotel
params = {'radius': 5000,
          'type': 'lodging',
          'key': g_key}

In [8]:
# Iterate through the Dataframe
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    # Add the latitude and longitude to the params dictionary as values to location key
    params['location'] = f'{lat},{lng}'
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        pass


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(3)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Hervey Bay,AU,86.00,clear sky,-25.30,152.85,Shelly Bay Resort
24,Thinadhoo,MV,82.49,overcast clouds,0.53,72.93,
40,Acapulco,MX,80.60,scattered clouds,16.86,-99.88,Crowne Plaza Acapulco


In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 4. Add cities to marker layer map
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# 5. Save and upload the new Dataframe
outfile = 'weather_data/challenge_data/WeatherPy_vacation.csv'
hotel_df.to_csv(outfile, index_label="City_ID")